# Visualization Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [9]:
import time
from contextlib import contextmanager


@contextmanager
def timer():
    """Helper function to time code within a context block."""
    start_time = time.perf_counter()
    yield
    duration = time.perf_counter() - start_time
    print(f"Ran in {duration:0.1f} secs")

In [22]:
import math
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from src.pipeline.flows.main_no_prefect import main_flow
from src.visualization.interactive_play_selector import create_interactive_play_selector

In [23]:
with timer():
    _ = main_flow(max_weeks=1, max_games=1, max_plays=1)

max_y = 1.9099999999999966
max_y = 1.8999999999999986
max_y = 1.8999999999999986
max_y = 1.8799999999999955
max_y = 1.8499999999999943
max_y = 1.7899999999999991
max_y = 1.7199999999999989
max_y = 1.5700000000000003
max_y = 1.3999999999999986
max_y = 1.1999999999999957
max_y = 0.9899999999999949
max_y = 0.7899999999999991
max_y = 0.5899999999999963
max_y = 0.1699999999999946
max_y = -0.23000000000000398
max_y = -0.6200000000000045
max_y = -0.980000000000004
max_y = -1.240000000000002
max_y = -1.4100000000000037
max_y = -1.5700000000000003
max_y = -1.740000000000002
max_y = -1.9000000000000057
max_y = -2.0500000000000043
max_y = -2.1700000000000017
max_y = -2.280000000000001
max_y = -2.3800000000000026
max_y = -2.469999999999999
max_y = -2.6099999999999994
max_y = -2.710000000000001
max_y = -4.18
max_y = -4.340000000000003
max_y = -4.510000000000005
max_y = -4.660000000000004
max_y = -5.600000000000001
max_y = -5.890000000000001
max_y = -6.150000000000006
max_y = -6.43
max_y = -6.710000

In [24]:
df_plays_all = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_tracking_display_all = pd.read_csv(f"{DIR}/data/outputs/tracking_display.csv")
df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

In [25]:
create_interactive_play_selector(
    df_plays_all,
    df_tracking_display_all,
    df_areas_all,
    continuous_update=True
)

interactive(children=(Dropdown(description='Week', options=(1,), value=1), Dropdown(description='Game ID', opt…